## *Setup*

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import csv
from scipy.stats import linregress

In [20]:
# Import of Census data and creation of a dataframe

census_data = "../project_one/census_income_data.csv"

census_data_income = pd.read_csv(census_data)
census_data_income.head()

,Table with column headers in rows 3 and 4,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,2018 Poverty and Median Household Income Estim...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Geographic Information,NaN,NaN,NaN,All Ages,NaN,NaN,NaN,NaN,NaN,...,NaN,Median Household Income,NaN,NaN,Age 0-4,NaN,NaN,NaN,NaN,NaN
2,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,...,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 0-4",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 0-4",90% CI Lower Bound,90% CI Upper Bound
3,00,000,US,United States,"41,852,315","41,619,366","42,085,264",13.1,13.0,13.2,...,17.2,"61,937","61,843","62,031","3,758,704","3,714,862","3,802,546",19.5,19.3,19.7
4,01,000,AL,Alabama,"801,758","785,668","817,848",16.8,16.5,17.1,...,23.7,"49,881","49,123","50,639","73,915","69,990","77,840",26.0,24.6,27.4


In [21]:
# Verification of request success of EPA API data
    
url = "https://enviro.epa.gov/enviro/efservice/SDW_VIOL_ENFORCEMENT/CSV"

response = requests.get(url).json
print(response)

<bound method Response.json of <Response [200]>>


In [22]:
# Request of EPA data from API CSV data and conversion to data frame

CSV_URL = "https://enviro.epa.gov/enviro/efservice/SDW_VIOL_ENFORCEMENT/CSV" 

#requesting csv from epa website decoding/reading as csv then converting to a list    
with requests.Session() as s:
    download = s.get(CSV_URL)

    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)
    
# Converting to dataframe    
epa_csv = pd.DataFrame(my_list)
epa_csv.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,SDW_VIOL_ENFORCEMENT.PWSID,SDW_VIOL_ENFORCEMENT.PWSNAME,SDW_VIOL_ENFORCEMENT.STATE,SDW_VIOL_ENFORCEMENT.COUNTYSERVED,SDW_VIOL_ENFORCEMENT.VIOID,SDW_VIOL_ENFORCEMENT.CCODE,SDW_VIOL_ENFORCEMENT.CNAME,SDW_VIOL_ENFORCEMENT.CTYPE,SDW_VIOL_ENFORCEMENT.VCODE,SDW_VIOL_ENFORCEMENT.VNAME,...,SDW_VIOL_ENFORCEMENT.VIOLMEASURE,SDW_VIOL_ENFORCEMENT.ENFACTIONTYPE,SDW_VIOL_ENFORCEMENT.ENFACTIONNAME,SDW_VIOL_ENFORCEMENT.ENFDATE,SDW_VIOL_ENFORCEMENT.COMPPERBEGINDATE,SDW_VIOL_ENFORCEMENT.COMPPERENDDATE,SDW_VIOL_ENFORCEMENT.SOURCES,SDW_VIOL_ENFORCEMENT.DEFINITION,SDW_VIOL_ENFORCEMENT.HEALTH_EFFECTS,
1,ID5160056,SPRINGDALE ACRES,ID,CASSIA,8251,2274,HEXACHLOROBENZENE,SOC,03,"Monitoring, Regular",...,Monitoring and Reporting,SIF,St Public Notif received,18-JUN-14,01-JAN-11,31-DEC-13,Discharge from metal refineries and agricultur...,"Hexachlorobenzene, a synthetic organic chemica...",Liver or kidney problems; reproductive difficu...,None
2,IN2493152,"G & S REALTY PARTNERS, LLC.",IN,MARION,6030,2992,Ethylbenzene,VOC,03,"Monitoring, Regular",...,Monitoring and Reporting,SIF,St Public Notif received,10-JUN-10,01-JAN-09,31-DEC-09,Discharge from petroleum refineries,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems,None
3,PA2450314,FEDEX FREIGHT EAST,PA,MONROE,1016111,2992,Ethylbenzene,VOC,03,"Monitoring, Regular",...,Monitoring and Reporting,SIA,St Violation/Reminder Notice,25-AUG-10,01-APR-10,30-JUN-10,Discharge from petroleum refineries,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems,None
4,WY5601199,KENNINGTON SPRINGS PIPELINE,WY,,2325,2046,Carbofuran,SOC,03,"Monitoring, Regular",...,Monitoring and Reporting,SOX,St Compliance achieved,24-OCT-11,01-JAN-08,31-DEC-10,Leaching of soil fumigant used on rice and alf...,Carbofuran is a white crystalline solid with a...,"Problems with blood, nervous system, or reprod...",None


## *Data Cleaning*

### Cleaning Census Data

In [23]:
# Rename Census data dataframe

census_try = census_data_income

In [24]:
# Removal of extraneaous rows

census_tryi=census_try.drop(census_try.index[0])
census_tryi.head()

,Table with column headers in rows 3 and 4,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
1,Geographic Information,NaN,NaN,NaN,All Ages,NaN,NaN,NaN,NaN,NaN,...,NaN,Median Household Income,NaN,NaN,Age 0-4,NaN,NaN,NaN,NaN,NaN
2,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,...,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 0-4",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 0-4",90% CI Lower Bound,90% CI Upper Bound
3,00,000,US,United States,"41,852,315","41,619,366","42,085,264",13.1,13.0,13.2,...,17.2,"61,937","61,843","62,031","3,758,704","3,714,862","3,802,546",19.5,19.3,19.7
4,01,000,AL,Alabama,"801,758","785,668","817,848",16.8,16.5,17.1,...,23.7,"49,881","49,123","50,639","73,915","69,990","77,840",26.0,24.6,27.4
5,01,001,AL,Autauga County,"7,587","6,334","8,840",13.8,11.5,16.1,...,23.9,"59,338","53,628","65,048",.,.,.,.,.,.


In [25]:
# Removal of extraneaous rows

census_tryin=census_tryi.drop(census_tryi.index[0])
census_tryin.head()

,Table with column headers in rows 3 and 4,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
2,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,...,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 0-4",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 0-4",90% CI Lower Bound,90% CI Upper Bound
3,00,000,US,United States,"41,852,315","41,619,366","42,085,264",13.1,13.0,13.2,...,17.2,"61,937","61,843","62,031","3,758,704","3,714,862","3,802,546",19.5,19.3,19.7
4,01,000,AL,Alabama,"801,758","785,668","817,848",16.8,16.5,17.1,...,23.7,"49,881","49,123","50,639","73,915","69,990","77,840",26.0,24.6,27.4
5,01,001,AL,Autauga County,"7,587","6,334","8,840",13.8,11.5,16.1,...,23.9,"59,338","53,628","65,048",.,.,.,.,.,.
6,01,003,AL,Baldwin County,"21,069","17,390","24,748",9.8,8.1,11.5,...,16.9,"57,588","54,437","60,739",.,.,.,.,.,.


In [26]:
# Duplication of first row as the dataframe header

census_tryin.rename(columns=census_tryin.iloc[0],inplace=True)

census_tryin.head()

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,...,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 0-4",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 0-4",90% CI Lower Bound,90% CI Upper Bound
2,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,...,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 0-4",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 0-4",90% CI Lower Bound,90% CI Upper Bound
3,00,000,US,United States,"41,852,315","41,619,366","42,085,264",13.1,13.0,13.2,...,17.2,"61,937","61,843","62,031","3,758,704","3,714,862","3,802,546",19.5,19.3,19.7
4,01,000,AL,Alabama,"801,758","785,668","817,848",16.8,16.5,17.1,...,23.7,"49,881","49,123","50,639","73,915","69,990","77,840",26.0,24.6,27.4
5,01,001,AL,Autauga County,"7,587","6,334","8,840",13.8,11.5,16.1,...,23.9,"59,338","53,628","65,048",.,.,.,.,.,.
6,01,003,AL,Baldwin County,"21,069","17,390","24,748",9.8,8.1,11.5,...,16.9,"57,588","54,437","60,739",.,.,.,.,.,.


In [28]:
# Removal of extraneaous rows

census_trying=census_tryin.drop(census_tryin.index[0])
census_trying.head()

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,...,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 0-4",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 0-4",90% CI Lower Bound,90% CI Upper Bound
3,00,000,US,United States,"41,852,315","41,619,366","42,085,264",13.1,13.0,13.2,...,17.2,"61,937","61,843","62,031","3,758,704","3,714,862","3,802,546",19.5,19.3,19.7
4,01,000,AL,Alabama,"801,758","785,668","817,848",16.8,16.5,17.1,...,23.7,"49,881","49,123","50,639","73,915","69,990","77,840",26.0,24.6,27.4
5,01,001,AL,Autauga County,"7,587","6,334","8,840",13.8,11.5,16.1,...,23.9,"59,338","53,628","65,048",.,.,.,.,.,.
6,01,003,AL,Baldwin County,"21,069","17,390","24,748",9.8,8.1,11.5,...,16.9,"57,588","54,437","60,739",.,.,.,.,.,.
7,01,005,AL,Barbour County,"6,788","5,662","7,914",30.9,25.8,36.0,...,45.9,"34,382","31,157","37,607",.,.,.,.,.,.


In [29]:
# Reset Index

census_trying.reset_index(drop=True)

,State FIPS Code,County FIPS Code,Postal Code,Name,"Poverty Estimate, All Ages",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, All Ages",90% CI Lower Bound,90% CI Upper Bound,...,90% CI Upper Bound,Median Household Income,90% CI Lower Bound,90% CI Upper Bound,"Poverty Estimate, Age 0-4",90% CI Lower Bound,90% CI Upper Bound,"Poverty Percent, Age 0-4",90% CI Lower Bound,90% CI Upper Bound
0,00,000,US,United States,"41,852,315","41,619,366","42,085,264",13.1,13.0,13.2,...,17.2,"61,937","61,843","62,031","3,758,704","3,714,862","3,802,546",19.5,19.3,19.7
1,01,000,AL,Alabama,"801,758","785,668","817,848",16.8,16.5,17.1,...,23.7,"49,881","49,123","50,639","73,915","69,990","77,840",26.0,24.6,27.4
2,01,001,AL,Autauga County,"7,587","6,334","8,840",13.8,11.5,16.1,...,23.9,"59,338","53,628","65,048",.,.,.,.,.,.
3,01,003,AL,Baldwin County,"21,069","17,390","24,748",9.8,8.1,11.5,...,16.9,"57,588","54,437","60,739",.,.,.,.,.,.
4,01,005,AL,Barbour County,"6,788","5,662","7,914",30.9,25.8,36.0,...,45.9,"34,382","31,157","37,607",.,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,56,037,WY,Sweetwater County,"3,540","2,804","4,276",8.4,6.7,10.1,...,11.2,"73,315","67,141","79,489",.,.,.,.,.,.
3190,56,039,WY,Teton County,"1,443","1,137","1,749",6.3,5.0,7.6,...,7.7,"99,087","88,142","110,032",.,.,.,.,.,.
3191,56,041,WY,Uinta County,"2,010","1,585","2,435",10.0,7.9,12.1,...,14.0,"63,401","56,812","69,990",.,.,.,.,.,.
3192,56,043,WY,Washakie County,918,700,"1,136",11.9,9.1,14.7,...,19.3,"55,190","49,263","61,117",.,.,.,.,.,.


In [30]:
# Creation of new data frame with relevant columns

new_census = census_tryin[['Postal Code','Name', 'Median Household Income']]

In [33]:
# Identification of naming conventions outside of "County"

bad_census = new_census[new_census['Name'].str.contains("County")==False]
bad_census

,Postal Code,Name,Median Household Income
2,Postal Code,Name,Median Household Income
3,US,United States,"61,937"
4,AL,Alabama,"49,881"
72,AK,Alaska,"74,912"
73,AK,Aleutians East Borough,"70,081"
...,...,...,...
3003,VA,Winchester city,"51,456"
3004,WA,Washington,"74,023"
3044,WV,West Virginia,"44,038"
3100,WI,Wisconsin,"60,779"


In [34]:
# Displaying all rows in this dataframe 

# pd.set_option('display.max_rows', bad_census.shape[0]+1)
# bad_census

In [32]:
# Extracting rows with county-level data

clean_census = new_census[new_census['Name'].str.contains("County|city|Borough|Census|Parish|District")]
clean_census.head()

,Postal Code,Name,Median Household Income
5,AL,Autauga County,"59,338"
6,AL,Baldwin County,"57,588"
7,AL,Barbour County,"34,382"
8,AL,Bibb County,"46,064"
9,AL,Blount County,"50,412"


In [47]:
# Reformatting County data to match format of EPA data for merge

clean_census['Name2'] = clean_census['Name'].str.replace(' County', '')
clean_census['NameUp'] = clean_census['Name2'].str.upper()

clean_census.head()

C:\Users\T\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\T\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Postal Code,Name,Median Household Income,Name2,NameUp
5,AL,Autauga County,"59,338",Autauga,AUTAUGA
6,AL,Baldwin County,"57,588",Baldwin,BALDWIN
7,AL,Barbour County,"34,382",Barbour,BARBOUR
8,AL,Bibb County,"46,064",Bibb,BIBB
9,AL,Blount County,"50,412",Blount,BLOUNT


In [50]:
# Removing and reordering extraneous columns

merge_census = clean_census[['Postal Code','NameUp','Median Household Income']]

merge_census.head()

,Postal Code,NameUp,Median Household Income
5,AL,AUTAUGA,"59,338"
6,AL,BALDWIN,"57,588"
7,AL,BARBOUR,"34,382"
8,AL,BIBB,"46,064"
9,AL,BLOUNT,"50,412"


In [51]:
# Reset Index

merge_census.reset_index(drop=True)

,Postal Code,NameUp,Median Household Income
0,AL,AUTAUGA,"59,338"
1,AL,BALDWIN,"57,588"
2,AL,BARBOUR,"34,382"
3,AL,BIBB,"46,064"
4,AL,BLOUNT,"50,412"
...,...,...,...
3136,WY,SWEETWATER,"73,315"
3137,WY,TETON,"99,087"
3138,WY,UINTA,"63,401"
3139,WY,WASHAKIE,"55,190"


### Cleaning EPA Data 

In [52]:
# Renaming EPA dataframe
epa_try =  epa_csv

In [53]:
# Duplication of first row as the dataframe header

epa_try.rename(columns=epa_try.iloc[0],inplace=True)

epa_try.head()

,SDW_VIOL_ENFORCEMENT.PWSID,SDW_VIOL_ENFORCEMENT.PWSNAME,SDW_VIOL_ENFORCEMENT.STATE,SDW_VIOL_ENFORCEMENT.COUNTYSERVED,SDW_VIOL_ENFORCEMENT.VIOID,SDW_VIOL_ENFORCEMENT.CCODE,SDW_VIOL_ENFORCEMENT.CNAME,SDW_VIOL_ENFORCEMENT.CTYPE,SDW_VIOL_ENFORCEMENT.VCODE,SDW_VIOL_ENFORCEMENT.VNAME,...,SDW_VIOL_ENFORCEMENT.VIOLMEASURE,SDW_VIOL_ENFORCEMENT.ENFACTIONTYPE,SDW_VIOL_ENFORCEMENT.ENFACTIONNAME,SDW_VIOL_ENFORCEMENT.ENFDATE,SDW_VIOL_ENFORCEMENT.COMPPERBEGINDATE,SDW_VIOL_ENFORCEMENT.COMPPERENDDATE,SDW_VIOL_ENFORCEMENT.SOURCES,SDW_VIOL_ENFORCEMENT.DEFINITION,SDW_VIOL_ENFORCEMENT.HEALTH_EFFECTS,
0,SDW_VIOL_ENFORCEMENT.PWSID,SDW_VIOL_ENFORCEMENT.PWSNAME,SDW_VIOL_ENFORCEMENT.STATE,SDW_VIOL_ENFORCEMENT.COUNTYSERVED,SDW_VIOL_ENFORCEMENT.VIOID,SDW_VIOL_ENFORCEMENT.CCODE,SDW_VIOL_ENFORCEMENT.CNAME,SDW_VIOL_ENFORCEMENT.CTYPE,SDW_VIOL_ENFORCEMENT.VCODE,SDW_VIOL_ENFORCEMENT.VNAME,...,SDW_VIOL_ENFORCEMENT.VIOLMEASURE,SDW_VIOL_ENFORCEMENT.ENFACTIONTYPE,SDW_VIOL_ENFORCEMENT.ENFACTIONNAME,SDW_VIOL_ENFORCEMENT.ENFDATE,SDW_VIOL_ENFORCEMENT.COMPPERBEGINDATE,SDW_VIOL_ENFORCEMENT.COMPPERENDDATE,SDW_VIOL_ENFORCEMENT.SOURCES,SDW_VIOL_ENFORCEMENT.DEFINITION,SDW_VIOL_ENFORCEMENT.HEALTH_EFFECTS,
1,ID5160056,SPRINGDALE ACRES,ID,CASSIA,8251,2274,HEXACHLOROBENZENE,SOC,03,"Monitoring, Regular",...,Monitoring and Reporting,SIF,St Public Notif received,18-JUN-14,01-JAN-11,31-DEC-13,Discharge from metal refineries and agricultur...,"Hexachlorobenzene, a synthetic organic chemica...",Liver or kidney problems; reproductive difficu...,None
2,IN2493152,"G & S REALTY PARTNERS, LLC.",IN,MARION,6030,2992,Ethylbenzene,VOC,03,"Monitoring, Regular",...,Monitoring and Reporting,SIF,St Public Notif received,10-JUN-10,01-JAN-09,31-DEC-09,Discharge from petroleum refineries,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems,None
3,PA2450314,FEDEX FREIGHT EAST,PA,MONROE,1016111,2992,Ethylbenzene,VOC,03,"Monitoring, Regular",...,Monitoring and Reporting,SIA,St Violation/Reminder Notice,25-AUG-10,01-APR-10,30-JUN-10,Discharge from petroleum refineries,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems,None
4,WY5601199,KENNINGTON SPRINGS PIPELINE,WY,,2325,2046,Carbofuran,SOC,03,"Monitoring, Regular",...,Monitoring and Reporting,SOX,St Compliance achieved,24-OCT-11,01-JAN-08,31-DEC-10,Leaching of soil fumigant used on rice and alf...,Carbofuran is a white crystalline solid with a...,"Problems with blood, nervous system, or reprod...",None


In [54]:
# Displaying EPA Data Columns for Reference

epa_try.columns

Index(['SDW_VIOL_ENFORCEMENT.PWSID', 'SDW_VIOL_ENFORCEMENT.PWSNAME',
       'SDW_VIOL_ENFORCEMENT.STATE', 'SDW_VIOL_ENFORCEMENT.COUNTYSERVED',
       'SDW_VIOL_ENFORCEMENT.VIOID', 'SDW_VIOL_ENFORCEMENT.CCODE',
       'SDW_VIOL_ENFORCEMENT.CNAME', 'SDW_VIOL_ENFORCEMENT.CTYPE',
       'SDW_VIOL_ENFORCEMENT.VCODE', 'SDW_VIOL_ENFORCEMENT.VNAME',
       'SDW_VIOL_ENFORCEMENT.VTYPE', 'SDW_VIOL_ENFORCEMENT.VIOLMEASURE',
       'SDW_VIOL_ENFORCEMENT.ENFACTIONTYPE',
       'SDW_VIOL_ENFORCEMENT.ENFACTIONNAME', 'SDW_VIOL_ENFORCEMENT.ENFDATE',
       'SDW_VIOL_ENFORCEMENT.COMPPERBEGINDATE',
       'SDW_VIOL_ENFORCEMENT.COMPPERENDDATE', 'SDW_VIOL_ENFORCEMENT.SOURCES',
       'SDW_VIOL_ENFORCEMENT.DEFINITION',
       'SDW_VIOL_ENFORCEMENT.HEALTH_EFFECTS', ''],
      dtype='object')

In [55]:
# Selection of pertinent columns

epa_data_info = epa_try[["SDW_VIOL_ENFORCEMENT.STATE", 
                         "SDW_VIOL_ENFORCEMENT.COUNTYSERVED", 
                         "SDW_VIOL_ENFORCEMENT.CNAME", 
                         "SDW_VIOL_ENFORCEMENT.SOURCES", 
                         "SDW_VIOL_ENFORCEMENT.DEFINITION", 
                         "SDW_VIOL_ENFORCEMENT.HEALTH_EFFECTS"]]

epa_data_info.head()

,SDW_VIOL_ENFORCEMENT.STATE,SDW_VIOL_ENFORCEMENT.COUNTYSERVED,SDW_VIOL_ENFORCEMENT.CNAME,SDW_VIOL_ENFORCEMENT.SOURCES,SDW_VIOL_ENFORCEMENT.DEFINITION,SDW_VIOL_ENFORCEMENT.HEALTH_EFFECTS
0,SDW_VIOL_ENFORCEMENT.STATE,SDW_VIOL_ENFORCEMENT.COUNTYSERVED,SDW_VIOL_ENFORCEMENT.CNAME,SDW_VIOL_ENFORCEMENT.SOURCES,SDW_VIOL_ENFORCEMENT.DEFINITION,SDW_VIOL_ENFORCEMENT.HEALTH_EFFECTS
1,ID,CASSIA,HEXACHLOROBENZENE,Discharge from metal refineries and agricultur...,"Hexachlorobenzene, a synthetic organic chemica...",Liver or kidney problems; reproductive difficu...
2,IN,MARION,Ethylbenzene,Discharge from petroleum refineries,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems
3,PA,MONROE,Ethylbenzene,Discharge from petroleum refineries,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems
4,WY,,Carbofuran,Leaching of soil fumigant used on rice and alf...,Carbofuran is a white crystalline solid with a...,"Problems with blood, nervous system, or reprod..."


In [56]:
# Removal of extraneous rows

epa_data_info = epa_data_info.drop(epa_data_info.index[0])
epa_data_info.head()

,SDW_VIOL_ENFORCEMENT.STATE,SDW_VIOL_ENFORCEMENT.COUNTYSERVED,SDW_VIOL_ENFORCEMENT.CNAME,SDW_VIOL_ENFORCEMENT.SOURCES,SDW_VIOL_ENFORCEMENT.DEFINITION,SDW_VIOL_ENFORCEMENT.HEALTH_EFFECTS
1,ID,CASSIA,HEXACHLOROBENZENE,Discharge from metal refineries and agricultur...,"Hexachlorobenzene, a synthetic organic chemica...",Liver or kidney problems; reproductive difficu...
2,IN,MARION,Ethylbenzene,Discharge from petroleum refineries,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems
3,PA,MONROE,Ethylbenzene,Discharge from petroleum refineries,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems
4,WY,,Carbofuran,Leaching of soil fumigant used on rice and alf...,Carbofuran is a white crystalline solid with a...,"Problems with blood, nervous system, or reprod..."
5,IN,WAYNE,Carbofuran,Leaching of soil fumigant used on rice and alf...,Carbofuran is a white crystalline solid with a...,"Problems with blood, nervous system, or reprod..."


In [57]:
# Renaming of Columns

epa_data_clean = epa_data_info.rename(columns={"SDW_VIOL_ENFORCEMENT.STATE": "State",
                              "SDW_VIOL_ENFORCEMENT.COUNTYSERVED": "County",
                               "SDW_VIOL_ENFORCEMENT.SOURCES": "Violation Source",
                              "SDW_VIOL_ENFORCEMENT.CNAME": "Chemical Name",
                              "SDW_VIOL_ENFORCEMENT.DEFINITION": "Chemical Definition",
                               "SDW_VIOL_ENFORCEMENT.HEALTH_EFFECTS": "Health Effects"})
epa_data_clean.head()

,State,County,Chemical Name,Violation Source,Chemical Definition,Health Effects
1,ID,CASSIA,HEXACHLOROBENZENE,Discharge from metal refineries and agricultur...,"Hexachlorobenzene, a synthetic organic chemica...",Liver or kidney problems; reproductive difficu...
2,IN,MARION,Ethylbenzene,Discharge from petroleum refineries,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems
3,PA,MONROE,Ethylbenzene,Discharge from petroleum refineries,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems
4,WY,,Carbofuran,Leaching of soil fumigant used on rice and alf...,Carbofuran is a white crystalline solid with a...,"Problems with blood, nervous system, or reprod..."
5,IN,WAYNE,Carbofuran,Leaching of soil fumigant used on rice and alf...,Carbofuran is a white crystalline solid with a...,"Problems with blood, nervous system, or reprod..."


In [58]:
# Reordering of Columns

epa_data_clean2 = epa_data_clean[['State', 'County', 'Violation Source', 'Chemical Name', 'Chemical Definition', 'Health Effects']]

epa_data_clean2.head()

,State,County,Violation Source,Chemical Name,Chemical Definition,Health Effects
1,ID,CASSIA,Discharge from metal refineries and agricultur...,HEXACHLOROBENZENE,"Hexachlorobenzene, a synthetic organic chemica...",Liver or kidney problems; reproductive difficu...
2,IN,MARION,Discharge from petroleum refineries,Ethylbenzene,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems
3,PA,MONROE,Discharge from petroleum refineries,Ethylbenzene,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems
4,WY,,Leaching of soil fumigant used on rice and alf...,Carbofuran,Carbofuran is a white crystalline solid with a...,"Problems with blood, nervous system, or reprod..."
5,IN,WAYNE,Leaching of soil fumigant used on rice and alf...,Carbofuran,Carbofuran is a white crystalline solid with a...,"Problems with blood, nervous system, or reprod..."


In [59]:
# Deletion of rows without county name

epa_data_clean3 = epa_data_clean2[epa_data_clean2.County != ""]

epa_data_clean3.head()

,State,County,Violation Source,Chemical Name,Chemical Definition,Health Effects
1,ID,CASSIA,Discharge from metal refineries and agricultur...,HEXACHLOROBENZENE,"Hexachlorobenzene, a synthetic organic chemica...",Liver or kidney problems; reproductive difficu...
2,IN,MARION,Discharge from petroleum refineries,Ethylbenzene,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems
3,PA,MONROE,Discharge from petroleum refineries,Ethylbenzene,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems
5,IN,WAYNE,Leaching of soil fumigant used on rice and alf...,Carbofuran,Carbofuran is a white crystalline solid with a...,"Problems with blood, nervous system, or reprod..."
6,MT,FLATHEAD,Leaching of soil fumigant used on rice and alf...,Carbofuran,Carbofuran is a white crystalline solid with a...,"Problems with blood, nervous system, or reprod..."


In [60]:
# Reset Index

epa_data_clean3.reset_index(drop=True)

,State,County,Violation Source,Chemical Name,Chemical Definition,Health Effects
0,ID,CASSIA,Discharge from metal refineries and agricultur...,HEXACHLOROBENZENE,"Hexachlorobenzene, a synthetic organic chemica...",Liver or kidney problems; reproductive difficu...
1,IN,MARION,Discharge from petroleum refineries,Ethylbenzene,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems
2,PA,MONROE,Discharge from petroleum refineries,Ethylbenzene,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems
3,IN,WAYNE,Leaching of soil fumigant used on rice and alf...,Carbofuran,Carbofuran is a white crystalline solid with a...,"Problems with blood, nervous system, or reprod..."
4,MT,FLATHEAD,Leaching of soil fumigant used on rice and alf...,Carbofuran,Carbofuran is a white crystalline solid with a...,"Problems with blood, nervous system, or reprod..."
...,...,...,...,...,...,...
70170,WV,WYOMING,Discharge from factories; leaching from gas st...,Benzene,"Benzene, a volatile organic chemical, is a cle...",Anemia; decrease in blood platelets; increased...
70171,NC,CASWELL,Discharge from factories; leaching from gas st...,Benzene,"Benzene, a volatile organic chemical, is a cle...",Anemia; decrease in blood platelets; increased...
70172,PA,SCHUYLKILL,Discharge from factories; leaching from gas st...,Benzene,"Benzene, a volatile organic chemical, is a cle...",Anemia; decrease in blood platelets; increased...
70173,VA,CULPEPER,Discharge from factories; leaching from gas st...,Benzene,"Benzene, a volatile organic chemical, is a cle...",Anemia; decrease in blood platelets; increased...


In [61]:
# Mergring dataframes based on State and County

epa_census = pd.merge(epa_data_clean3,merge_census, how='left', left_on=['State','County'], right_on = ['Postal Code','NameUp'])

epa_census.head()

,State,County,Violation Source,Chemical Name,Chemical Definition,Health Effects,Postal Code,NameUp,Median Household Income
0,ID,CASSIA,Discharge from metal refineries and agricultur...,HEXACHLOROBENZENE,"Hexachlorobenzene, a synthetic organic chemica...",Liver or kidney problems; reproductive difficu...,ID,CASSIA,"51,804"
1,IN,MARION,Discharge from petroleum refineries,Ethylbenzene,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems,IN,MARION,"48,409"
2,PA,MONROE,Discharge from petroleum refineries,Ethylbenzene,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems,PA,MONROE,"63,318"
3,IN,WAYNE,Leaching of soil fumigant used on rice and alf...,Carbofuran,Carbofuran is a white crystalline solid with a...,"Problems with blood, nervous system, or reprod...",IN,WAYNE,"45,130"
4,MT,FLATHEAD,Leaching of soil fumigant used on rice and alf...,Carbofuran,Carbofuran is a white crystalline solid with a...,"Problems with blood, nervous system, or reprod...",MT,FLATHEAD,"54,382"


In [62]:
# Reording and selection of pertinent columns

epa_census_df = epa_census[["State",
                            "County",
                            "Median Household Income",
                            "Violation Source",
                            "Chemical Name",
                            "Chemical Definition",
                            "Health Effects"]]

epa_census_df.head()

,State,County,Median Household Income,Violation Source,Chemical Name,Chemical Definition,Health Effects
0,ID,CASSIA,"51,804",Discharge from metal refineries and agricultur...,HEXACHLOROBENZENE,"Hexachlorobenzene, a synthetic organic chemica...",Liver or kidney problems; reproductive difficu...
1,IN,MARION,"48,409",Discharge from petroleum refineries,Ethylbenzene,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems
2,PA,MONROE,"63,318",Discharge from petroleum refineries,Ethylbenzene,Ethylbenzene is a colorless organic liquid wit...,Liver or kidneys problems
3,IN,WAYNE,"45,130",Leaching of soil fumigant used on rice and alf...,Carbofuran,Carbofuran is a white crystalline solid with a...,"Problems with blood, nervous system, or reprod..."
4,MT,FLATHEAD,"54,382",Leaching of soil fumigant used on rice and alf...,Carbofuran,Carbofuran is a white crystalline solid with a...,"Problems with blood, nervous system, or reprod..."


## *Data Analysis*